In [1]:
import pandas as pd
import numpy as np
ltr_dev = pd.read_json('2020-11-29_16:56:08_b0444fd2-328d-11eb-a5d8-bfed4aec1d90/output.json')
dev_qrel = pd.read_csv('collections/msmarco-passage/qrels.dev.small.tsv', sep="\t",
                       names=["qid", "q0", "pid", "rel"], 
                       usecols=['qid', 'pid', 'rel'])                                                                                         
ltr_dev = ltr_dev.merge(dev_qrel, left_on=['qid', 'pid'], right_on=['qid', 'pid'], how='left')                                                                                                           
ltr_dev['rel'] = ltr_dev['rel'].fillna(0)                                                                                                                                             
ltr_dev = ltr_dev.sort_values(['qid','pid']).set_index(['qid','pid'])
ltr_dev.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 6974598 entries, (2, 55860) to (1102400, 8830447)
Data columns (total 2 columns):
 #   Column  Dtype  
---  ------  -----  
 0   score   float64
 1   rel     float64
dtypes: float64(2)
memory usage: 335.9 MB


In [2]:
import pandas as pd
import numpy as np
bert_dev = pd.read_csv('../pygaggle/runs/run.monobert.ans_entire.dev.trec', sep="\t", 
                  names=['qid','q0','pid','rank','score','tag'],
                  usecols=['qid', 'pid', 'score'])
dev_qrel = pd.read_csv('collections/msmarco-passage/qrels.dev.small.tsv', sep="\t", 
                       names=["qid", "q0", "pid", "rel"], 
                       usecols=['qid', 'pid', 'rel'])                                                                                         
bert_dev = bert_dev.merge(dev_qrel, left_on=['qid', 'pid'], right_on=['qid', 'pid'], how='left')                                                                                                           
bert_dev['rel'] = bert_dev['rel'].fillna(0)                                                                                                                                            
bert_dev = bert_dev.sort_values(['qid','pid']).set_index(['qid','pid'])
bert_dev.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 6974598 entries, (2, 55860) to (1102400, 8830447)
Data columns (total 2 columns):
 #   Column  Dtype  
---  ------  -----  
 0   score   float64
 1   rel     float64
dtypes: float64(2)
memory usage: 335.9 MB


In [5]:
from tqdm import tqdm
def eval_mrr(dev_data):                                                                                                                                                                                  
    score_tie_counter = 0                                                                                                                                                                                
    score_tie_query = set()                                                                                                                                                                              
    MRR = []                                                                                                                                                                                             
    for qid, group in tqdm(dev_data.groupby('qid')):                                                                                                                                                     
        group = group.reset_index()                                                                                                                                                                      
        rank = 0                                                                                                                                                                                         
        prev_score = None                                                                                                                                                                                
        assert len(group['pid'].tolist()) == len(set(group['pid'].tolist()))                                                                                                                             
        # stable sort is also used in LightGBM                                                                                                                                                           
                                                                                                                                                                                                         
        for t in group.sort_values('score', ascending=False, kind='mergesort').itertuples():                                                                                                             
            if prev_score is not None and abs(t.score - prev_score) < 1e-8:                                                                                                                              
                score_tie_counter += 1                                                                                                                                                                   
                score_tie_query.add(qid)                                                                                                                                                                 
            prev_score = t.score                                                                                                                                                                         
            prev_pid = t.pid                                                                                                                                                                             
            rank += 1                                                                                                                                                                                    
            if t.rel>0:                                                                                                                                                                                  
                MRR.append(1.0/rank)                                                                                                                                                                     
                break                                                                                                                                                                                    
            elif rank == 10 or rank == len(group):
                MRR.append(0.)
                break

    score_tie = f'score_tie occurs {score_tie_counter} times in {len(score_tie_query)} queries'
    print(score_tie)
    mrr_10 = np.mean(MRR).item()
    print(f'MRR@10:{mrr_10} with {len(MRR)} queries')
    return {'score_tie': score_tie, 'mrr_10': mrr_10}

In [6]:
eval_mrr(ltr_dev)

100%|██████████| 6980/6980 [00:12<00:00, 538.90it/s]

score_tie occurs 895 times in 647 queries
MRR@10:0.21159025787965619 with 6980 queries


{'score_tie': 'score_tie occurs 895 times in 647 queries',
 'mrr_10': 0.21159025787965619}

In [7]:
eval_mrr(bert_dev)

100%|██████████| 6980/6980 [00:12<00:00, 545.30it/s]

score_tie occurs 106 times in 103 queries
MRR@10:0.37926735116205024 with 6980 queries


{'score_tie': 'score_tie occurs 106 times in 103 queries',
 'mrr_10': 0.37926735116205024}

In [8]:
merge_dev = ltr_dev.join(bert_dev,lsuffix='ltr',rsuffix='bert')
assert (merge_dev['relltr'] == merge_dev['relbert']).all()
merge_dev = merge_dev.rename(columns={"relltr": "rel"})
del merge_dev['relbert']
merge_dev.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 6974598 entries, (2, 55860) to (1102400, 8830447)
Data columns (total 3 columns):
 #   Column     Dtype  
---  ------     -----  
 0   scoreltr   float64
 1   rel        float64
 2   scorebert  float64
dtypes: float64(3)
memory usage: 709.1 MB


In [18]:
def genBertRerank(merge_dev, cutoff=100):
    bert_rerank = []
    for qid, group in tqdm(merge_dev.groupby('qid')):                                                                                                                                                     
        group = group.reset_index()                                                                                                                                                                      
        rank = 0
        temp = []
        for t in group.sort_values('scoreltr', ascending=False, kind='mergesort').itertuples():                                                                                                                                                                                                                                                                                     
            rank += 1
            if rank <= cutoff:
                temp.append((qid, t.pid, t.scorebert, rank, t.rel))
            else:
                break
        bert_rerank.extend(sorted(temp, key=lambda x:x[2], reverse=True))
    return pd.DataFrame(bert_rerank, 
                        columns=['qid','pid','score','rank','rel']
                       ).set_index(['qid','pid'])

In [21]:
for i in range(100,1001,100):
    eval_mrr(genBertRerank(merge_dev, i))

100%|██████████| 6980/6980 [00:12<00:00, 568.95it/s]


score_tie occurs 107 times in 104 queries
MRR@10:0.36238891162959935 with 6980 queries


100%|██████████| 6980/6980 [00:12<00:00, 573.23it/s]


score_tie occurs 107 times in 102 queries
MRR@10:0.37100764087870103 with 6980 queries


100%|██████████| 6980/6980 [00:13<00:00, 519.78it/s]


...

score_tie occurs 110 times in 106 queries
MRR@10:0.37886592077136494 with 6980 queries


100%|██████████| 6980/6980 [00:13<00:00, 499.45it/s]


score_tie occurs 105 times in 103 queries
MRR@10:0.37900594669577475 with 6980 queries


100%|██████████| 6980/6980 [00:14<00:00, 484.89it/s]

score_tie occurs 106 times in 103 queries
MRR@10:0.3792195956701687 with 6980 queries
